# Custom Auto Augment for Object Detection

Objective:
* Create custom data augment policies for object detection
* Apply transformation on image, plot and save 


Input:
* Image to apply augmentations
* Bounding Box Co-ordinates
* Required fields: image_name, xmin, xmax, ymin, ymax, height, width, defect_type
* BBox order should be  [ymin, xmin, ymax, xmax]


Output:
* Save augment image in the specified working directory

When will this module be used?
* This is used to plot and view various data augments on the image.


In [ ]:
from auto_augment import autoaugment_utils_custom as aug
import tensorflow as tf
import numpy as np
from PIL import Image
import cv2
import tensorflow as tf
import numpy as np
import pandas as pd
from PIL import Image
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt
%matplotlib inline

### Read image and plot defects

In [ ]:
# Read and view input image
path = r'E:\PoC_Work/'
input_img = Image.open(path+"14e90eb54.jpg")
input_img


In [ ]:
# Read annotations 
df = pd.read_csv("AA_image_data.csv")
print(df.image_name.nunique())
df.head()

In [ ]:
# Plot annotations 
img_names = df.image_name.unique()
input_path=r'E:\PoC_Work/'
#output_path=r'E:\Parsytec\App_audited\plotted_img/'
img_names = df.image_name.unique()
for im in img_names:
    img =cv2.imread(input_path+im)
    df5 = df.loc[df['image_name']==im]
    for index2, row2 in df5.iterrows():
        cv2.rectangle(img, (row2['xmin'],row2['ymin']), (row2['xmax'],row2['ymax']), (0, 255, 0), 2)
plt.figure(figsize = (30,15))
plt.imshow(img)        

### Create Custom Policies

In [ ]:
 custom_policy =   [[('Color', 0.0, 6), ('Cutout', 0.6, 8), ('Sharpness', 0.4, 8)], [('Sharpness', 0.4, 2)],
     [('AutoContrast', 0.8, 2)], [('AutoContrast', 0.4, 6), ('ShearX_BBox', 0.8, 8), ('Brightness', 0.0, 10)],
     [('SolarizeAdd', 0.2, 6), ('Contrast', 0.0, 10), ('AutoContrast', 0.6, 0)],
     [('Cutout', 0.2, 0), ('Solarize', 0.8, 8), ('Color', 1.0, 4)],
     [('TranslateY_BBox', 0.0, 4), ('Equalize', 0.6, 8), ('Solarize', 0.0, 10)], [('TranslateY_BBox', 0.2, 2)],
     [('Cutout', 0.8, 8), ('Brightness', 0.8, 8), ('Cutout', 0.2, 2)], [('Color', 0.8, 4)],
     [('BBox_Cutout', 1.0, 4), ('Cutout', 0.2, 8)], [('Equalize', 0.6, 6)],
     [('AutoContrast', 0.4, 2), ('Brightness', 0.2, 2)],
     [('Solarize', 0.4, 6), ('SolarizeAdd', 0.2, 10)],
     [('SolarizeAdd', 0.2, 8), ('Equalize', 0.2, 4)]]

### Apply Custom Policies

In [ ]:

bboxes=[]
for im in img_names:
    img =cv2.imread(input_path+im)
    df5 = df.loc[df['image_name']==im]
    for index2, row2 in df5.iterrows(): # normalize
        bboxes.append([row2['ymin']/row2['height'],row2['xmin']/row2['width'],
                       row2['ymax']/row2['height'],row2['xmax']/row2['width']])


# bboxes = [[ymin1,xmin1,ymax1,xmax1],[ymin2,xmin2,ymax2,xmax2]]

# Convert image to tensor
image_tensor = tf.convert_to_tensor(img)
# Create array of bounding box co-ordinates
data_np = np.asarray(bboxes, np.float32)
# Convert bounding box list to tensor
data_tf = tf.convert_to_tensor(data_np, np.float32)

for i in range(len(custom_policy)):
    if len(custom_policy[i]) > 1:
        #print("in first->",policy_v2[i])
        for j in range(len(custom_policy[i])):
            print("********** Applying Policy -->",custom_policy[i][j])
            image, res_bboxes = aug.distort_image_with_autoaugment(image_tensor, data_tf,'custom_v0',custom_policy[i][j])

            sess = tf.Session()
            with sess.as_default():
                r_img = image.eval()

            sess = tf.Session()
            with sess.as_default():
                aug_box = res_bboxes.eval()

            cv2.rectangle(r_img, (int(aug_box[0][1]*width),int(aug_box[0][0]*height)), 
                          (int(aug_box[0][3]*width),int(aug_box[0][2]*height)), (0, 255, 0), 2)

            cv2.rectangle(r_img, (int(aug_box[1][1]*width),int(aug_box[1][0]*height)), 
                          (int(aug_box[1][3]*width),int(aug_box[1][2]*height)), (0, 0, 255), 2)
            plt.figure(figsize = (30,15))
            cv2.imwrite("Kaggle_Custom_Policies\custom_policy_v0/"+str(custom_policy[i][j])+".png",r_img)
            #plt.imshow(r_img)